## K-TACC을 사용하여 다수의 주체가 포함된 뉴스 데이터 증강

In [1]:
from BERT_augmentation import BertAugmentation
from adverb_augmentation import AdverbAugmentation
import pandas as pd

---
### 라이브러리 초기화 및 함수 정의
- `BERT`, `Adverb`를 사용하여 한국어 텍스트 데이터 증강
- 출처: 문맥을 고려한 한국어 텍스트 데이터 증강 (Korean Text Augmentation Considering Context, K-TACC) (https://github.com/kyle-bong/K-TACC)
- 감정 레이블을 예측을 했을 때, 여러 주체가 포함된 뉴스 데이터에서는 **감정 레이블 예측이 부정확**
- 이러한 문제를 해결하기 위해, 예측 모델의 추가적인 Fine-tuning을 위해 **직접 선별한 100여 개의 데이터 샘플을 증강**하여 새로운 데이터셋을 생성

In [2]:
# 초기화
bert = BertAugmentation()
adverb = AdverbAugmentation()

# 증강 함수들
random_masking_insertion = bert.random_masking_insertion
adverb_gloss_replacement = adverb.adverb_gloss_replacement

---
### 데이터 증강 과정
- **랜덤 마스킹 및 삽입 (`random_masking_insertion`)**: 원본 텍스트에서 일부단어를 무작위로 마스킹하거나 삽입하여 문장 변형
- **부사 교체 (`adverb_gloss_replacement`)**: 텍스트에서 부사를 동의어로 교체하여 문장의 의미를 유지하면서 다양한 형태로 변환
- 텍스트 변환 과정에서 원문의 의미가 지나치게 변하지 않도록 변환 강도를 조정하여 데이터 증강

In [3]:
# 데이터 증강 함수 (원본 텍스트, 증강 횟수, 변환 강도)
def augment_text(text, num_augments=5, ratio=0.2):
    augmented_texts = []
    for _ in range(num_augments):
        # 증강 단계 적용
        aug_text = random_masking_insertion(sentence=text[:512], ratio=ratio)
        aug_text = adverb_gloss_replacement(sentence=aug_text)
        augmented_texts.append(aug_text)
    return augmented_texts

---
### 데이터 증강 적용

- **CSV 파일 읽기**: 여러 개의 주체가 포함된 뉴스 데이터 파일 read
- **데이터 증강**: 각 문장에 대해 5번의 증강을 적용하여 새로운 데이터셋을 생성
- **중복 제거**: 증강된 데이터에서 중복된 문장을 제거하여 데이터의 다양성을 유지
- **기업명 강조**: 모델이 학습할 때 해당 기업에 더욱 집중할 수 있도록 텍스트에서 특정 기업명인 `SK하이닉스`와 `sk하이닉스`를 `<SK하이닉스>`로 변환.
- **결과 저장**: 증강된 데이터를 새로운 CSV 파일로 저장
- **증강 결과**: 94 ==> 536

In [4]:
def generate_augmented_dataset(input_file, output_file, num_augments=5, ratio=0.2):
    # 여러개의 주체가 존재하는 뉴스 데이터 CSV 파일 불러오기
    df = pd.read_csv(input_file, header=None, encoding='cp949')
    print(f"원본 데이터 수: {len(df)}")

    # 컬럼명 설정 (0번 뉴스 요약, 1번 감정 레이블)
    df.columns = ["description", "label"]

    # 증강 데이터 생성
    augmented_data = []
    for _, row in df.iterrows():
        text, label = row["description"], row["label"]
        augmented_texts = augment_text(text, num_augments=num_augments, ratio=ratio)

        # 원본 텍스트와 증강된 텍스트들을 하나의 리스트에 추가
        augmented_data.append([text, label])    # 원본
        for aug_text in augmented_texts:        # 증강 된 데이터
            augmented_data.append([aug_text, label])

    # 새로운 데이터프레임 생성
    augmented_df = pd.DataFrame(augmented_data, columns=["description", "label"])

    # 중복 제거
    augmented_df = augmented_df.drop_duplicates(subset=["description"])
    
    # "SK하이닉스", "sk하이닉스" 텍스트 변환
    augmented_df = augmented_df.applymap(lambda x: x.replace("SK하이닉스", "<SK하이닉스>") if isinstance(x, str) else x)
    augmented_df = augmented_df.applymap(lambda x: x.replace("sk하이닉스", "<SK하이닉스>") if isinstance(x, str) else x)

    # 결과 저장
    augmented_df.to_csv(output_file, index=False, header=False)
    print(f"증강 된 데이터 셋 {output_file}. 증강 된 전체 데이터 수: {len(augmented_df)}")

In [5]:
# 파일 경로 설정
input_csv = "input.csv"
output_csv = "augmented_subject_focus_data.csv"

# 증강된 데이터 생성
generate_augmented_dataset(input_csv, output_csv, num_augments=5, ratio=0.2)

원본 데이터 수: 92
증강 된 데이터 셋 augmented_subject_focus_data.csv. 증강 된 전체 데이터 수: 536
